In [9]:
import warnings
warnings.filterwarnings('default')

In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import forestci as fci
import pickle5 as pickle

In [10]:
with open('G:/cassidy/MODISLANDSAT_new/calbiomeDictLAI.pkl', "rb") as fh:
    data = pickle.load(fh)

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from 'C:\\Users\\casli\\Anaconda3\\envs\\py37\\lib\\site-packages\\pandas\\_libs\\internals.cp37-win_amd64.pyd'>

In [8]:
calbiomeDictLAI = pd.read_pickle('G:/cassidy/MODISLANDSAT_new/calbiomeDictLAI.pkl')
# valbiomeDictLAI = pd.read_pickle('G:/cassidy/MODISLANDSAT_new/valbiomeDictLAI.pkl')

ValueError: unsupported pickle protocol: 5

In [ ]:
dfcal = calbiomeDictLAI[6]['DF']
dfval = valbiomeDictLAI[6]['DF'].sample(50)

regressors = ['red','NIR','cosSZA','cosVZA','cosSA']
response = 'LAI'

X_train = dfcal[regressors]
y_train = dfcal[response]

X_test = dfval[regressors]
y_test = dfval[response]

In [ ]:
rf = RandomForestRegressor(n_estimators=100,min_samples_leaf=10,max_features=5,bootstrap=True,random_state=0,verbose=0,n_jobs=-1,max_leaf_nodes=10000)
rf.fit(X_train, y_train)

## Scikit-garden

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from skgarden import RandomForestQuantileRegressor

In [ ]:
rfqr = RandomForestQuantileRegressor(n_estimators=100,min_samples_leaf=10,max_features=5,bootstrap=True,random_state=0,verbose=0,n_jobs=-1,max_leaf_nodes=10000)

In [ ]:
y_true_all = []
lower = []
upper = []

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (
        X[train_index], X[test_index], y[train_index], y[test_index])

    rfqr.set_params(max_features=X_train.shape[1] // 3)
    rfqr.fit(X_train, y_train)
    y_true_all = np.concatenate((y_true_all, y_test))
    upper = np.concatenate((upper, rfqr.predict(X_test, quantile=98.5)))
    lower = np.concatenate((lower, rfqr.predict(X_test, quantile=2.5)))

interval = upper - lower
sort_ind = np.argsort(interval)
y_true_all = y_true_all[sort_ind]
upper = upper[sort_ind]
lower = lower[sort_ind]
mean = (upper + lower) / 2

# Center such that the mean of the prediction interval is at 0.0
y_true_all -= mean
upper -= mean
lower -= mean

In [ ]:
print(upper[0])

In [ ]:
print(lower[0])

In [ ]:
print(y_true_all[0])

## forestci

In [ ]:
# retreive mpg data from machine learning library
dfcal = calbiomeDictLAI[1]['DF'].sample(50000)

regressors = ['red','NIR','cosSZA','cosVZA','cosSA']
response = 'LAI'
# separate mpg data into predictors and outcome variable
X_train = dfcal[regressors]
y_train = dfcal[response]

# split mpg data into training and test set
# mpg_X_train, mpg_X_test, mpg_y_train, mpg_y_test = xval.train_test_split(
#     mpg_X,
#     mpg_y,
#     test_size=0.25,
#     random_state=42)

# Create RandomForestRegressor
forest = RandomForestRegressor(n_estimators=10,min_samples_leaf=10,max_features=5,bootstrap=True,random_state=0,verbose=0,n_jobs=-1,max_leaf_nodes=10000)
forest.fit(X_train ,np.array(y_train).ravel())


In [ ]:
dfval = valbiomeDictLAI[1]['DF'].sample(50)
X_test = dfval[regressors]
y_test = dfval[response]

y_hat = forest.predict(X_test)

# Plot predicted MPG without error bars
plt.scatter(y_test, y_hat)
plt.plot([0, 70], [0, 70], 'k--')
plt.xlabel('Reference LAI')
plt.ylabel('Predicted LAI')
plt.show()

# Calculate the variance
V_IJ_unbiased = fci.random_forest_error(forest, X_train, X_test)

# Plot error bars for predicted MPG using unbiased variance
plt.errorbar(y_test, y_hat, yerr=np.sqrt(V_IJ_unbiased), fmt='o')
plt.plot([0, 70], [0, 70], 'k--')
plt.xlabel('Reference LAI')
plt.ylabel('Predicted LAI')
plt.show()

In [ ]:
V_IJ_unbiased

In [ ]:
# retreive mpg data from machine learning library
mpg_data = fetch_openml(data_id=196)

# separate mpg data into predictors and outcome variable
mpg_X = mpg_data["data"]
mpg_y = mpg_data["target"]

# remove rows where the data is nan
not_null_sel = np.where(mpg_X.isna().sum(axis=1).values == 0)
mpg_X = mpg_X.values[not_null_sel]
mpg_y = mpg_y.values[not_null_sel]

# split mpg data into training and test set
mpg_X_train, mpg_X_test, mpg_y_train, mpg_y_test = xval.train_test_split(
    mpg_X,
    mpg_y,
    test_size=0.25,
    random_state=42)

# Create RandomForestRegressor
n_trees = 2000
mpg_forest = RandomForestRegressor(n_estimators=n_trees, random_state=42)
mpg_forest.fit(mpg_X_train, mpg_y_train)
mpg_y_hat = mpg_forest.predict(mpg_X_test)

# Plot predicted MPG without error bars
plt.scatter(mpg_y_test, mpg_y_hat)
plt.plot([5, 45], [5, 45], 'k--')
plt.xlabel('Reported MPG')
plt.ylabel('Predicted MPG')
plt.show()

# Calculate the variance
mpg_V_IJ_unbiased = fci.random_forest_error(mpg_forest, mpg_X_train,
                                            mpg_X_test)

# Plot error bars for predicted MPG using unbiased variance
plt.errorbar(mpg_y_test, mpg_y_hat, yerr=np.sqrt(mpg_V_IJ_unbiased), fmt='o')
plt.plot([5, 45], [5, 45], 'k--')
plt.xlabel('Reported MPG')
plt.ylabel('Predicted MPG')
plt.show()

## method 3

In [ ]:
def pred_ints(model, X, percentile=95):
    err_down = []
    err_up = []
    for x in range (len(X_test)):
        preds = []
        for pred in model.estimators_:
            preds.append(pred.predict(np.array(X.iloc[x]).reshape(1, -1)))
        err_down.append(np.percentile(preds, (100 - percentile) / 2. ))
        err_up.append(np.percentile(preds, 100 - (100 - percentile) / 2.))
    return err_down, err_up

In [ ]:
err_down, err_up = pred_ints(rf, X_test, percentile=90)
 
truth = y_test
correct = 0.
for i, val in enumerate(truth):
    if err_down[i] <= val <= err_up[i]:
        correct += 1
print (correct/len(truth))

In [ ]:
y_hat = rf.predict(X_test)

In [ ]:
plt.errorbar(y_test, y_hat, yerr=np.subtract(err_up, err_down), ls='None', lw=1)
plt.scatter(y_test, y_hat, s=15)
plt.plot([0, 70], [0, 70], 'k--')
plt.xlabel('Reference LAI')
plt.ylabel('Predicted LAI')
plt.show()